In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]

In [8]:
#  MoM FFBL Weekly Rankings Replacement
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390"""
cur = DatabaseCursor(PATH, options="-c search_path=dev")
matchups = cur.copy_data_from_postgres(matchups_query)
cur = DatabaseCursor(PATH, options="-c search_path=dev")
teams = cur.copy_data_from_postgres(teams_query)
matchups = matchups[(matchups['is_playoffs'] == 0) & (matchups['is_consolation'] == 0) & (matchups['game_id'] >= 390)]
matchups_a = matchups.copy()
matchups_b = matchups.copy()
matchups_b_cols = list(matchups_b.columns)
rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"
matchups_b.rename(columns=rename_columns, inplace=True)
matchups = pd.concat([matchups_a, matchups_b])
matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)
matchups.reset_index(drop=True, inplace=True)
matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)
matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)
matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)
matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)
matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]
weeks = list(matchups["week"].unique())
regular_season = matchups[matchups["week"] == 1]
for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )
reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(['game_id'])["2pt_total"].rank(method="min", ascending=False).astype(int),
)
regular_season.insert(
    4,
    "pts_rank",
    regular_season.groupby(['game_id'])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "projected_pts_rank",
    regular_season.groupby(['game_id'])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    0,
    "tuple",
    regular_season[["2pt_rank", "pts_rank"]]
    .apply(tuple, axis=1),
)
regular_season.insert(
    0,
    "overall_rank",
    regular_season.groupby(['game_id'])['tuple'].rank(method="min", ascending=True).astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)
team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_y"]
regular_season.drop(["team_a_team_key", "name", "nickname", "tuple", "game_id_x", "game_id_y"], axis=1, inplace=True)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(["overall_rank", "2pt_total", 'team_a_points_total'], ascending=[True, False, False], inplace=True)
regular_season

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390


,game_id,overall_rank,manager,team_name,2pt_total,2pt_rank,team_a_points_total,pts_rank,team_a_project_points_total,projected_pts_rank,wins,losses,week_1,team_a_points,team_a_projected_points,win_loss,2pt_system,week_2,team_a_points_2,team_a_projected_points_2,win_loss_2,2pt_system_2,game_id_2,week_3,team_a_points_3,team_a_projected_points_3,win_loss_3,2pt_system_3,game_id_3,week_4,team_a_points_4,team_a_projected_points_4,win_loss_4,2pt_system_4,game_id_4,week_5,team_a_points_5,team_a_projected_points_5,win_loss_5,2pt_system_5,game_id_5,week_6,team_a_points_6,team_a_projected_points_6,win_loss_6,2pt_system_6,game_id_6,week_7,team_a_points_7,team_a_projected_points_7,win_loss_7,2pt_system_7,game_id_7,week_8,team_a_points_8,team_a_projected_points_8,win_loss_8,2pt_system_8,game_id_8,week_9,team_a_points_9,team_a_projected_points_9,win_loss_9,2pt_system_9,game_id_9,week_10,team_a_points_10,team_a_projected_points_10,win_loss_10,2pt_system_10,game_id_10,week_11,team_a_points_11,team_a_projected_points_11,win_loss_11,2pt_system_11,game_id_11,week_12,team_a_points_12,team_a_projected_points_12,win_loss_12,2pt_system_12,game_id_12,week_13,team_a_points_13,team_a_projected_points_13,win_loss_13,2pt_system_13,game_id_13,week_14,team_a_points_14,team_a_projected_points_14,win_loss_14,2pt_system_14,game_id_14,week_15,team_a_points_15,team_a_projected_points_15,win_loss_15,2pt_system_15,game_id_15,team_key
20,406,1,Tim,Fantasy Sidelines,21.0,1,1725.32,4,1871.51,2,11,4,1,125.26,125.12,W,1,2,120.24,126.71,W,0,406,3,136.88,126.08,W,1,406,4,116.76,125.55,W,1,406,5,150.64,128.29,W,1,406,6,135.28,128.82,W,1,406,7,101.84,116.76,W,0,406,8,112.20,129.83,W,1,406,9,66.30,123.52,L,0,406,10,118.20,121.88,W,1,406,11,123.58,125.75,L,1,406,12,94.94,113.05,L,0,406,13,120.52,127.57,W,1,406,14.0,100.12,129.66,L,0.0,406.0,15.0,102.56,122.92,W,1.0,406.0,406.l.103661.t.1
17,399,1,Pat,Push Ups in Prison,20.0,1,1645.78,1,1584.08,1,10,3,1,111.54,126.07,L,0,2,146.18,120.34,L,1,399,3,169.30,129.33,W,1,399,4,125.54,129.50,W,1,399,5,151.40,127.05,W,1,399,6,123.20,113.59,W,1,399,7,122.70,123.70,W,1,399,8,140.04,119.64,W,1,399,9,98.38,119.36,W,0,399,10,85.38,117.24,L,0,399,11,115.02,119.69,W,1,399,12,139.08,120.53,W,1,399,13,118.02,118.04,W,1,399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.l.12514.t.7
4,390,1,Shinn,Runningback U (JS),20.0,1,1622.86,1,1589.91,1,10,3,1,144.06,121.13,W,1,2,95.58,125.81,L,0,390,3,104.28,126.96,W,0,390,4,114.78,112.84,W,1,390,5,161.14,120.20,W,1,390,6,102.54,116.03,L,0,390,7,107.42,123.39,W,1,390,8,123.28,119.37,W,1,390,9,146.42,127.19,W,1,390,10,131.52,123.52,W,1,390,11,121.28,116.12,W,1,390,12,150.36,126.01,W,1,390,13,120.20,131.34,L,1,390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,390.l.215172.t.4
24,406,2,ryan,King of the Hill,20.0,2,1854.08,1,1957.11,1,10,5,1,131.32,130.74,W,1,2,139.62,130.89,W,1,406,3,129.50,128.95,L,1,406,4,142.52,131.24,W,1,406,5,127.98,140.79,W,0,406,6,153.08,126.25,W,1,406,7,93.34,135.20,L,0,406,8,109.00,138.16,L,0,406,9,83.24,131.55,L,0,406,10,127.14,133.26,W,1,406,11,101.30,115.81,L,0,406,12,139.16,114.31,W,1,406,13,129.96,133.79,W,1,406,14.0,126.32,135.28,W,1.0,406.0,15.0,120.60,130.89,W,1.0,406.0,406.l.103661.t.4
15,399,2,Greg,Krispy Kareem 🍩,20.0,1,1529.10,3,1559.82,3,11,2,1,117.48,116.11,W,1,2,121.80,117.01,W,1,399,3,125.44,116.52,W,1,399,4,97.92,118.86,W,0,399,5,95.52,116.19,W,0,399,6,96.08,117.72,W,0,399,7,149.58,124.52,W,1,399,8,101.26,124.07,L,0,399,9,116.40,122.01,L,1,399,10,139.06,122.07,W,1,399,11,121.28,117.46,W,1,399,12,117.38,130.08,W,1,399,13,129.90,117.20,W,1,399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.l.12514.t.5
6,390,2,Greg,Keenan & Bell (GF),15.0,2,1412.26,7,1532.88,2,8,5,1,83.66,116.81,L,0,2,121.62,114.28,W,1,390,3,143.50,124.68,W,1,390,4,91.74,114.39,L,0,390,5,108.42,114.56,W,1,390,6,107.90,123.24,W,1,390,7,99.36,123.51,L,0,390,8,132.30,114.02,W,1,390,9,114.34,109.28,L,0,390,10,127.66,125.31,W,1,390,11,89.72,122.30,W,0,390,12,90.12,107.46,L,0,390,13,101.92,123.04,W